In [1]:
from src.utils import load_dataset, get_path
from src.adapter import adapt_to_dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from typing import List

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
data_train_exaction = load_dataset(get_path('data', 'npy', 'audio_data_train.npy'))
data_test_exaction = load_dataset(get_path('data', 'npy', 'audio_data_test.npy'))

In [3]:
data_train = pd.read_csv('data/metadata_train_challenge.csv')
data_test = pd.read_csv('data/metadata_private_test.csv')
data_train = adapt_to_dataset(data_train, data_train_exaction)
data_test = adapt_to_dataset(data_test, data_test_exaction)

In [4]:
# def conver_to_train(data_list_exaction, data_list):
#     data = data_list_exaction.tolist()
#     data = pd.DataFrame(data)
#     data['assessment_result'] = [(i.assessment_result) for i in data_list.samples]
#     data = data.drop(['name', 'sampling_rate'],axis=1)
#     genre_list = data.iloc[:, -1]
#     encoder = LabelEncoder()
#     y = encoder.fit_transform(genre_list)
#     p = [i['allfeat'] for i in data.features]
#     data_1 = pd.DataFrame(p)
#     scaler = StandardScaler()
#     X = scaler.fit_transform(np.array(data_1.iloc[:, :-1], dtype = float))
#     X_train = X
#     y_train = y
#     return X_train, y_train


In [ ]:
def conver_to_train(data_list_exaction, data_list):
    data = data_list_exaction.tolist()
    data = pd.DataFrame(data)
    data['assessment_result'] = [(i.assessment_result) for i in data_list.samples]
    data = data.drop(['name', 'sampling_rate'],axis=1)
    genre_list = data.iloc[:, -1]
    encoder = LabelEncoder()
    y = encoder.fit_transform(genre_list)
    p = [i['allfeat'] for i in data.features]
    data_1 = pd.DataFrame(p)
    scaler = StandardScaler()
    X = scaler.fit_transform(np.array(data_1.iloc[:, :-1], dtype = float))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test


In [5]:
X_train, y_train = conver_to_train(data_train_exaction,data_train)
X_test, y_test = conver_to_train(data_test_exaction,data_test)



In [6]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [7]:

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])   

In [8]:
history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=128)

Epoch 1/100
10/10 [==============================] - 23s 4ms/step - loss: 2.2689 - accuracy: 0.1294
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 2.1512 - accuracy: 0.5844
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 2.0054 - accuracy: 0.6047
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 1.8296 - accuracy: 0.6088
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 1.6118 - accuracy: 0.6145
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 1.3756 - accuracy: 0.6082
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 1.1686 - accuracy: 0.6090
Epoch 8/100
10/10 [==============================] - 0s 4ms/step - loss: 1.0021 - accuracy: 0.6173
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 0.8779 - accuracy: 0.6108
Epoch 10/100
10/10 [==============================] - 0s 3ms/step - loss: 0.8133 - accuracy: 0.5988
Epoch 11

In [9]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

15/15 [==============================] - 1s 2ms/step - loss: 0.4829 - accuracy: 0.9867
test_acc:  0.9866666793823242


In [10]:
y_predict = model.predict(X_test)

In [11]:
resutl = pd.read_csv('data/private_test_sample_submissions.csv')
resutl['assessment_result'] = y_predict
resutl.to_csv('results.csv', index = False)